In [2]:
from backtesting import Backtest
from backtesting.test import EURUSD, GOOG
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

In [3]:
EURUSD.head()

,Open,High,Low,Close,Volume
2017-04-19 09:00:00,1.07160,1.07220,1.07083,1.07219,1413
2017-04-19 10:00:00,1.07214,1.07296,1.07214,1.07260,1241
2017-04-19 11:00:00,1.07256,1.07299,1.07170,1.07192,1025
2017-04-19 12:00:00,1.07195,1.07280,1.07195,1.07202,1460
2017-04-19 13:00:00,1.07200,1.07230,1.07045,1.07050,1554


In [4]:
EURUSD.tail()

,Open,High,Low,Close,Volume
2018-02-07 11:00:00,1.23390,1.23548,1.23386,1.23501,2203
2018-02-07 12:00:00,1.23501,1.23508,1.23342,1.23422,2325
2018-02-07 13:00:00,1.23422,1.23459,1.23338,1.23372,2824
2018-02-07 14:00:00,1.23374,1.23452,1.23238,1.23426,4065
2018-02-07 15:00:00,1.23427,1.23444,1.22904,1.22904,6143


In [7]:
bt = Backtest(EURUSD, SMA, cash = 10000, exclusive_orders=True)
output=bt.run()
bt.plot
print(output)

TypeError: `strategy` must be a Strategy sub-type